In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [5]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [7]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [8]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [10]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [14]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
embedded_sequences = keras.layers.core.Reshape((max_sequence_length,300,1))(embedded_sequences)  ## reshape layer
x1 = Conv2D(2, (2, 300), activation='relu')(embedded_sequences)                 ## conv 1
x1 = GlobalMaxPooling2D()(x1)                                                   ## max 1
x2 = Conv2D(2, (3, 300), activation='relu')(embedded_sequences)                 ## conv 2
x2 = GlobalMaxPooling2D()(x2)                                                   ## max 2
x3 = Conv2D(2, (4, 300), activation='relu')(embedded_sequences)                 ## conv 3
x3 = GlobalMaxPooling2D()(x3)                                                   ## max 3
x  = keras.layers.concatenate([x1, x2, x3])                                     ## concatenate
preds = Dense(2, activation='softmax')(x)                                       ## softmax layer

model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from keras.utils import plot_model
plot_model(model, to_file='/content/drive/My Drive/Sarcasm Detection/Plots/CNN.png')

In [16]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/15
5002/5002 [==============================] - 4s 853us/step - loss: 0.3901 - acc: 0.8311
Epoch 2/15
5002/5002 [==============================] - 1s 158us/step - loss: 0.2162 - acc: 0.9208
Epoch 3/15
5002/5002 [==============================] - 1s 147us/step - loss: 0.1611 - acc: 0.9464
Epoch 4/15
5002/5002 [==============================] - 1s 155us/step - loss: 0.1253 - acc: 0.9620
Epoch 5/15
5002/5002 [==============================] - 1s 148us/step - loss: 0.0991 - acc: 0.9712
Epoch 6/15
5002/5002 [==============================] - 1s 151us/step - loss: 0.0788 - acc: 0.9808
Epoch 7/15
5002/5002 [==============================] - 1s 147us/step - loss: 0.0626 - acc: 0.9862
Epoch 8/15
5002/5002 [==============================] - 1s 143us/step - loss: 0.0495 - acc: 0.9918
Epoch 9/15
5002/5002 [==============================] - 1s 142us/step - loss: 0.03

In [17]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

1667/1667 [==============================] - 0s 142us/step
test loss is: 0.2469389635729299
test accuracy is:  0.9250149970005999


In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1 = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return f1

model.compile(loss='categorical_crossentropy',
          optimizer= "rmsprop",
          metrics=[f1])

In [19]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
5002/5002 [==============================] - 1s 181us/step - loss: 0.0059 - f1: 0.9994
Epoch 2/15
5002/5002 [==============================] - 1s 136us/step - loss: 0.0042 - f1: 0.9996
Epoch 3/15
5002/5002 [==============================] - 1s 137us/step - loss: 0.0030 - f1: 0.9998
Epoch 4/15
5002/5002 [==============================] - 1s 134us/step - loss: 0.0023 - f1: 0.9998
Epoch 5/15
5002/5002 [==============================] - 1s 139us/step - loss: 0.0017 - f1: 0.9998
Epoch 6/15
5002/5002 [==============================] - 1s 135us/step - loss: 0.0013 - f1: 0.9998
Epoch 7/15
5002/5002 [==============================] - 1s 134us/step - loss: 0.0010 - f1: 0.9998
Epoch 8/15
5002/5002 [==============================] - 1s 145us/step - loss: 8.3465e-04 - f1: 0.9998
Epoch 9/15
5002/5002 [==============================] - 1s 144us/step - loss: 6.9363e-04 - f1: 0.9998
Epoch 10/15
5002/5002 [==============================] - 1s 140us/step - loss: 5.9225e-04 - f1: 0.9998
Epoch 1

In [20]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

1667/1667 [==============================] - 0s 90us/step
test loss is: 0.46634478003236757
test f1 score is:  0.9232153031521382


In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 300)      6178800     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 34, 300, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 33, 1, 2)     1202        reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (